In [1]:
// Ventanas de tiempo

import java.sql.Timestamp
import org.apache.spark.sql.Row
import spark.implicits._

val lineas = spark.readStream
    .format("socket")
    .option("host", "localhost")
    .option("port", 9999)
    .option("includeTimestamp", true)
    .load()

Intitializing Scala interpreter ...

Spark Web UI available at http://10.0.2.15:4040
SparkContext available as 'sc' (version = 2.4.6, master = local[*], app id = local-1616221309098)
SparkSession available as 'spark'


import java.sql.Timestamp
import org.apache.spark.sql.Row
import spark.implicits._
lineas: org.apache.spark.sql.DataFrame = [value: string, timestamp: timestamp]


In [4]:
val palabras = lineas.as[(String, Timestamp)].flatMap( line =>
                    line._1.split(" ").map(word => (word, line._2)))
                    .toDF("palabra", "timestamp")

//val windowCount = palabras.groupBy(
//                    window($"timestamp", "10 seconds", "5 seconds"),
//                    $"palabra")
//                    .count()

// La ventana de arriba pero con watermark
val windowCount = palabras
                    .withWatermark("timestamp", "15 seconds")
                    .groupBy(
                    window($"timestamp", "10 seconds", "5 seconds"),
                    $"palabra")
                    .count()

val query = windowCount.writeStream.outputMode("update")
            .format("console")
            .option("truncate", false)
            .start()

query.awaitTermination(90000)

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----+
|window|palabra|count|
+------+-------+-----+
+------+-------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+--------+-----+
|window                                    |palabra |count|
+------------------------------------------+--------+-----+
|[2021-03-20 06:32:20, 2021-03-20 06:32:30]|hola    |1    |
|[2021-03-20 06:32:15, 2021-03-20 06:32:25]|amigos  |1    |
|[2021-03-20 06:32:25, 2021-03-20 06:32:35]|bernardo|1    |
|[2021-03-20 06:32:20, 2021-03-20 06:32:30]|amigos  |1    |
|[2021-03-20 06:32:15, 2021-03-20 06:32:25]|hola    |1    |
|[2021-03-20 06:32:20, 2021-03-20 06:32:30]|bernardo|1    |
+------------------------------------------+--------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------

palabras: org.apache.spark.sql.DataFrame = [palabra: string, timestamp: timestamp]
windowCount: org.apache.spark.sql.DataFrame = [window: struct<start: timestamp, end: timestamp>, palabra: string ... 1 more field]
query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@73e8e143
res2: Boolean = false


-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+-------+-----+
|window                                    |palabra|count|
+------------------------------------------+-------+-----+
|[2021-03-20 06:33:40, 2021-03-20 06:33:50]|amigos |1    |
|[2021-03-20 06:33:35, 2021-03-20 06:33:45]|amigos |1    |
+------------------------------------------+-------+-----+



In [5]:
query.stop()